In [561]:
import pandas as pd
import numpy as np

In [562]:
combined_data_file = "../../data/Data-Jul16/combined_jul16.csv"
data_2020_file = "../../data/Data-Jul16/F20_jul05.csv"

In [563]:
data = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")

In [564]:
data_2020 = pd.read_csv(data_2020_file, encoding = "ISO-8859-1")
data_2016 = data.loc[data['YEAR'] == 2016]
data_2012 = data.loc[data['YEAR'] == 2012]
data_2008 = data.loc[data['YEAR'] == 2008]

year_df_dict = {2020:data_2020, 2016:data_2016, 2012:data_2012, 2008:data_2008}

In [565]:
pd.set_option('max_columns', None)

In [566]:
data.sample(10)

,YEAR_FIPS,YEAR,STATE_FIPS,COUNTY_FIPS,STATE,COUNTY,REP_CANDIDATE,DEM_CANDIDATE,REP_VOTES,DEM_VOTES,COUNTY_TOTALVOTES,WINNING_CANDIDATE,WINNING_PARTY,WINNING_PARTY_BINARY,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,MARGIN_VICTORY
14470,200839125,2008,39,125,Ohio,Paulding County,John McCain,Barack Obama,5317,4165,9769.0,John McCain,republican,1,0,0,7.1,0,25,13,92,98,376,428,34,44,5,0,9993,9742,19735,9727,9460,5443,2752,2691,4518,2296,2222,5866,2966,2900,3908,1728,2180,0.001267,0.000659,0.004662,0.004966,0.019052,0.021687,0.001723,0.002230,0.000253,0.000000,0.492881,0.479351,0.506359,0.493641,0.275804,0.228933,0.297238,0.198024,-0.117924
646,201212009,2012,12,9,Florida,Brevard County,Mitt Romney,Barack Obama,159300,122993,285559.0,Mitt Romney,republican,1,1,1,9.4,46125,6890,4928,29038,27268,22616,21664,1158,1242,332,334,277432,266540,543972,233728,226613,120557,62416,58141,113022,57247,55775,162993,79474,83519,147400,67403,79997,0.012666,0.009059,0.053381,0.050128,0.041576,0.039826,0.002129,0.002283,0.000610,0.000614,0.429669,0.416589,0.510012,0.489988,0.221624,0.207772,0.299635,0.270970,-0.127144
7226,200022097,2000,22,97,Louisiana,St. Landry Parish,George W. Bush,Al Gore,15449,18067,34151.0,Al Gore,democrat,0,0,0,6.5,22233,96,83,19788,17227,367,420,74,61,7,5,45772,41890,87662,25581,24295,28626,14564,14062,22057,10429,11628,21444,10323,11121,15535,6574,8961,0.001095,0.000947,0.225731,0.196516,0.004187,0.004791,0.000844,0.000696,0.000080,0.000057,0.291814,0.277144,0.522142,0.477858,0.326550,0.251614,0.244621,0.177215,0.076660
8636,200048051,2000,48,51,Texas,Burleson County,George W. Bush,Al Gore,3542,2235,5866.0,George W. Bush,republican,1,1,-1,4.0,23560,19,11,1363,1133,1150,1261,52,35,1,3,8449,8000,16449,6948,6752,4877,2488,2389,3846,1906,1940,4276,2101,2175,3450,1505,1945,0.001155,0.000669,0.082862,0.068880,0.069913,0.076661,0.003161,0.002128,0.000061,0.000182,0.422396,0.410481,0.513648,0.486352,0.296492,0.233814,0.259955,0.209739,-0.222809
15360,20085131,2008,5,131,Arkansas,Sebastian County,John McCain,Barack Obama,28637,13673,43212.0,John McCain,republican,1,0,-1,5.0,35382,2588,2474,4135,3913,6388,7171,1237,1270,86,98,62644,60302,122946,52869,50949,34610,17689,16921,32894,16660,16234,33789,16591,17198,21653,9362,12291,0.021050,0.020123,0.033633,0.031827,0.051958,0.058326,0.010061,0.010330,0.000699,0.000797,0.430018,0.414401,0.509525,0.490475,0.281506,0.267548,0.274828,0.176118,-0.346293
10575,200427085,2004,27,85,Minnesota,McLeod County,George W. Bush,John Kerry,11407,6712,18412.0,George W. Bush,republican,1,0,1,4.6,33579,125,119,48,80,809,845,37,31,10,12,17889,17652,35541,17573,17311,10607,5427,5180,8972,4639,4333,9501,4807,4694,6461,2779,3682,0.003517,0.003348,0.001351,0.002251,0.022762,0.023775,0.001041,0.000872,0.000281,0.000338,0.494443,0.487071,0.503334,0.496666,0.298444,0.252441,0.267325,0.181790,-0.254997
2172,201236045,2012,36,45,New York,Jefferson County,Mitt Romney,Barack Obama,16435,16231,35708.0,Mitt Romney,republican,1,1,0,9.9,0,897,731,2369,3818,2752,3569,303,349,167,172,57118,59476,116594,51951,52912,32951,16885,16066,36205,19663,16542,28996,14588,14408,18442,8340,10102,0.007693,0.006270,0.020318,0.032746,0.023603,0.030610,0.002599,0.002993,0.001432,0.001475,0.445572,0.453814,0.489888,0.510112,0.282613,0.310522,0.248692,0.158173,-0.005713
7,20121015,2012,1,15,Alabama,Calhoun County,Mitt Romney,Barack Obama,30278,15511,46257.0,Mitt Romney,republican,

In [567]:
## some cleanup..

data.isnull().sum()

YEAR_FIPS           0
YEAR                0
STATE_FIPS          0
COUNTY_FIPS         0
STATE               0
                   ..
TOT_POP_LESS19%     0
TOT_POP_20to39%     0
TOT_POP_40to59%     0
TOT_POP_Above60%    0
MARGIN_VICTORY      0
Length: 64, dtype: int64

In [568]:
data.isna().sum()

YEAR_FIPS           0
YEAR                0
STATE_FIPS          0
COUNTY_FIPS         0
STATE               0
                   ..
TOT_POP_LESS19%     0
TOT_POP_20to39%     0
TOT_POP_40to59%     0
TOT_POP_Above60%    0
MARGIN_VICTORY      0
Length: 64, dtype: int64

In [569]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15733 entries, 0 to 15732
Data columns (total 64 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   YEAR_FIPS              15733 non-null  int64  
 1   YEAR                   15733 non-null  int64  
 2   STATE_FIPS             15733 non-null  int64  
 3   COUNTY_FIPS            15733 non-null  int64  
 4   STATE                  15733 non-null  object 
 5   COUNTY                 15733 non-null  object 
 6   REP_CANDIDATE          15733 non-null  object 
 7   DEM_CANDIDATE          15733 non-null  object 
 8   REP_VOTES              15733 non-null  int64  
 9   DEM_VOTES              15733 non-null  int64  
 10  COUNTY_TOTALVOTES      15733 non-null  float64
 11  WINNING_CANDIDATE      15733 non-null  object 
 12  WINNING_PARTY          15733 non-null  object 
 13  WINNING_PARTY_BINARY   15733 non-null  int64  
 14  HOUSE_WINNING_BINARY   15733 non-null  int64  
 15  SE

# cleaning/wrangling...

In [570]:
from sklearn.model_selection import train_test_split

In [571]:
def train_test_split_by_year_2(X, year, train_filter=False, cols=None):
    
    X = data.drop("WINNING_PARTY_BINARY", axis=1)
    #y = data[["WINNING_PARTY_BINARY", "TRAIN"]]
    y = data[["WINNING_PARTY_BINARY"]]

    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
    
    if (year == None):
        #then we're done.
        if (cols == None):
            return X_train, X_test, y_train, y_test
        else:
            return X_train[cols], X_test[cols], y_train, y_test
    
    filter_year =  X['YEAR'] == year
    not_filter_year =  X['YEAR'] != year
    
    X_test = X_test[filter_year]
    
    if (cols != None):
        X_test = X_test[cols]
    
    print('X_test: {}'.format(X_test.shape))
    
    y_test = y_test[filter_year]
    y_test = y_test.drop("TRAIN", axis=1)
    print('y_test: {}'.format(y_test.shape))

    X_train = X_train[not_filter_year]

    if (train_filter == True):
        X_train = X_train.query('TRAIN == 1')
        X_train = X_train.reset_index(drop=True)
        
    X_train.drop("TRAIN", axis=1)
    
    if (cols != None):
        X_train = X_train[cols]
   
    y_train = y_train[not_filter_year]
    
    if (train_filter == True):
        y_train = y_train.query('TRAIN == 1')
        y_train = y_train.reset_index(drop=True)
        
    y_train = y_train.drop("TRAIN", axis=1)
    
    print('X_train: {}'.format(X_train.shape))
    print('y_train: {}'.format(y_train.shape))

    return X_train, X_test, y_train, y_test

In [572]:
def train_test_split_by_year(X, y, year, cols=None):
    
    if year != None:
        year_filter =  X['YEAR'] < year
        X = X[year_filter]
        y = y[year_filter]
        
    X = X.drop('YEAR', axis=1)
    y = np.delete(y, 1, axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    if (cols == None):
        return X_train, X_test, y_train, y_test
    else:
        return X_train[cols], X_test[cols], y_train, y_test

### Configure Year###

In [573]:
year = 2016
target_df = year_df_dict.get(year)

### Experiment 1 - Train with the data as is -- i.e without balancing training data. 

In [574]:
data = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")

In [575]:
data['REP_VOTES%'] = data['REP_VOTES'] / data['COUNTY_TOTALVOTES']
data['DEM_VOTES%'] = data['DEM_VOTES'] / data['COUNTY_TOTALVOTES']
data.drop(['REP_VOTES', 'DEM_VOTES'], axis=1, inplace=True)


In [576]:
#add: REP_VOTES, DEM_VOTES, MARGIN_VICTORY
data.drop(['STATE_FIPS', 'COUNTY_FIPS', 'REP_VOTES%', 'DEM_VOTES%', 'MARGIN_VICTORY', 'WINNING_CANDIDATE', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)

In [577]:
data

,YEAR_FIPS,YEAR,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%
0,20121001,2012,23932.0,1,1,1,6.900000,33766,296,201,5152,4591,615,693,131,119,17,28,28084,26670,54754,22052,21357,16046,8188,7858,13845,6759,7086,15464,7577,7887,9399,4146,5253,0.005406,0.003671,0.094094,0.083848,0.011232,0.012657,0.002393,0.002173,0.000310,0.000511,0.402747,0.390054,0.512912,0.487088,0.293056,0.252858,0.282427,0.171659
1,20121003,2012,85338.0,1,1,1,7.500000,64282,833,551,8913,8421,3614,4450,670,660,55,73,93493,89618,183111,81752,78718,46410,23528,22882,42149,21095,21054,51691,25005,26686,42861,19990,22871,0.004549,0.003009,0.048675,0.045988,0.019737,0.024302,0.003659,0.003604,0.000300,0.000399,0.446461,0.429892,0.510581,0.489419,0.253453,0.230183,0.282293,0.234071
2,20121005,2012,11509.0,0,1,1,11.500000,64168,59,59,6111,6786,599,747,73,88,17,29,12826,14504,27330,6482,7414,6620,3373,3247,7401,4461,2940,7618,4124,3494,5691,2546,3145,0.002159,0.002159,0.223600,0.248299,0.021917,0.027333,0.002671,0.003220,0.000622,0.001061,0.237175,0.271277,0.469301,0.530699,0.242225,0.270801,0.278741,0.208233
3,20121007,2012,8420.0,1,1,1,8.500000,69624,10,12,1972,3069,110,300,32,36,5,26,10608,12264,22872,8491,9034,5691,3004,2687,6475,3809,2666,6426,3505,2921,4280,1946,2334,0.000437,0.000525,0.086219,0.134182,0.004809,0.013116,0.001399,0.001574,0.000219,0.001137,0.371240,0.394981,0.463799,0.536201,0.248820,0.283097,0.280955,0.187128
4,20121009,2012,24006.0,1,1,1,6.900000,60178,82,48,390,424,2081,2582,173,177,25,36,28988,28385,57373,27995,27415,15550,7963,7587,13866,7017,6849,15869,7941,7928,12088,5464,6624,0.001429,0.000837,0.006798,0.007390,0.036271,0.045004,0.003015,0.003085,0.000436,0.000627,0.487947,0.477838,0.505255,0.494745,0.271033,0.241682,0.276594,0.210691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15728,201246113,2012,3145.0,0,1,1,5.706686,0,1,3,1,9,82,95,5898,5864,2,4,6246,6220,12466,300,285,6114,3136,2978,3333,1622,1711,2113,1041,1072,906,421,485,0.000080,0.000241,0.000080,0.000722,0.006578,0.007621,0.473127,0.470399,0.000160,0.000321,0.024065,0.022862,0.501043,0.498957,0.490454,0.267367,0.169501,0.072678
15729,201646113,2016,2905.0,0,1,1,5.706686,0,1,4,1,6,119,116,6200,6101,1,4,6546,6447,12993,291,271,6183,3116,3067,3434,1701,1733,2366,1178,1188,1010,452,558,0.000077,0.000308,0.000077,0.000462,0.009159,0.008928,0.477180,0.469561,0.000077,0.000308,0.022397,0.020857,0.503810,0.496190,0.475872,0.264296,0.182098,0.077734
15730,2000102,2000,30000.0,1,1,1,14.200000,25000,4,10,3,1,156,160,6630,6445,0,3,6913,6723,13636,213,211,5930,3019,2911,3722,1828,1894,2777,1363,1414,1207,513,694,0.000293,0.000733,0.000220,0.000073,0.011440,0.011734,0.486213,0.472646,0.000000,0.000220,0.015620,0.015474,0.506967,0.493033,0.434878,0.272954,0.203652,0.088516
15731,2004102,2004,30000.0,1,1,1,14.200000,25000,4,10,3,1,156,160,6630,6445,0,3,6913,6723,13636,213,211,5930,3019,2911,3722,1828,1894,2777,1363,1414,1207,513,694,0.000293,0.000733,0.000220,0.000073,0.011440,0.011734,0.486213,0.472646,0.000000,0.000220,0.015620,0.015474,0.506967,0.493033,0.434878,0.272954,0.203652,0.088516


In [578]:
#X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year)

X = data.drop(['WINNING_PARTY_BINARY'], axis=1)
y = data[['WINNING_PARTY_BINARY', 'YEAR']].values

X_train, X_test, y_train, y_test = train_test_split_by_year(X, y, year)

In [579]:
from sklearn.linear_model import LogisticRegression

In [580]:
model1 = LogisticRegression()

In [584]:
model1.fit(X_train, y_train)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [585]:
X_train.columns

Index(['YEAR_FIPS', 'COUNTY_TOTALVOTES', 'HOUSE_WINNING_BINARY',
       'SENATE_WINNING_BINARY', 'UNEMPLOYMENT_RATE', 'AVG_WAGE_SALARY',
       'AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE',
       'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE', 'TOT_FEMALE',
       'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19',
       'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39',
       'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59',
       'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60',
       'TOT_MALE_Above60', 'TOT_FEMALE_Above60', 'AA_FEMALE%', 'AA_MALE%',
       'BA_FEMALE%', 'BA_MALE%', 'H_FEMALE%', 'H_MALE%', 'IA_FEMALE%',
       'IA_MALE%', 'NA_FEMALE%', 'NA_MALE%', 'WA_FEMALE%', 'WA_MALE%',
       'TOT_FEMALE%', 'TOT_MALE%', 'TOT_POP_LESS19%', 'TOT_POP_20to39%',
       'TOT_POP_40to59%', 'TOT_POP_Above60%'],
      dtype='object')

In [586]:
predictions = model1.predict(X_test)

In [587]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

In [588]:
print("Experiment#1: {:.3f}%".format( accuracy_score(y_test, predictions) * 100))

Experiment#1: 78.157%


In [589]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.71      0.15      0.25       907
           1       0.79      0.98      0.87      2870

   micro avg       0.78      0.78      0.78      3777
   macro avg       0.75      0.57      0.56      3777
weighted avg       0.77      0.78      0.72      3777



In [590]:
from sklearn.metrics import confusion_matrix, classification_report

In [591]:
confusion_matrix(y_test, predictions)

array([[ 139,  768],
       [  57, 2813]])

# Test with Random Forest Classifier

In [592]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [593]:
print(classification_report(y_test, rfc_pred))

              precision    recall  f1-score   support

           0       0.81      0.62      0.70       907
           1       0.89      0.95      0.92      2870

   micro avg       0.87      0.87      0.87      3777
   macro avg       0.85      0.79      0.81      3777
weighted avg       0.87      0.87      0.87      3777



### Experiment 2 - Balance training data 

In [99]:
data = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")

In [100]:
data.drop(['STATE_FIPS', 'COUNTY_FIPS', 'REP_VOTES', 'DEM_VOTES', 'MARGIN_VICTORY', 'WINNING_CANDIDATE', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)

In [101]:
### retest with Training=1 data.. 
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year, True)

In [102]:
X_train.shape

(11013, 52)

In [103]:
X_test.shape

(4720, 52)

In [104]:
model2 = LogisticRegression()

In [105]:
model2.fit(X_train, y_train)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [106]:
predictions = model2.predict(X_test)

In [107]:
print("Experiment#2: {:.3f}%".format( accuracy_score(y_test, predictions) * 100))

Experiment#2: 81.758%


In [108]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.80      0.19      0.31      1005
           1       0.82      0.99      0.89      3715

   micro avg       0.82      0.82      0.82      4720
   macro avg       0.81      0.59      0.60      4720
weighted avg       0.82      0.82      0.77      4720



In [109]:
confusion_matrix(y_test, predictions)

array([[ 191,  814],
       [  47, 3668]])

### Experiment 3 - Expr2 + remove demographics absolute values

In [619]:
data = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")

In [620]:
data.drop([ 'STATE_FIPS', 'COUNTY_FIPS', 'WINNING_CANDIDATE', 'REP_VOTES', 'DEM_VOTES', 'MARGIN_VICTORY', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)

In [621]:
data.drop(['AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE', 'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE' , 'TOT_FEMALE', 'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19', 'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39', 'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59', 'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60', 'TOT_MALE_Above60', 'TOT_FEMALE_Above60'] , axis=1, inplace=True)

In [401]:
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year, True)

X_test: (986, 25)


/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


KeyError: "['TRAIN'] not found in axis"

In [402]:
model3 = LogisticRegression()

In [403]:
model3.fit(X_train, y_train)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [404]:
predictions = model3.predict(X_test)

In [117]:
print("Experiment#3: {:.3f}%".format( accuracy_score(y_test, predictions) * 100))

Experiment#3: 80.636%


In [118]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.69      0.17      0.27      1005
           1       0.81      0.98      0.89      3715

   micro avg       0.81      0.81      0.81      4720
   macro avg       0.75      0.57      0.58      4720
weighted avg       0.79      0.81      0.76      4720



In [613]:
X = data.drop('WINNING_PARTY_BINARY', axis=1)
y = data[['WINNING_PARTY_BINARY', 'YEAR']].values

X_train, X_test, y_train, y_test = train_test_split_by_year(X, y, year)

In [614]:
rfc3 = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

rfc_pred3= rfc3.predict(X_test)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


In [615]:
print(classification_report(y_test, rfc_pred3))

              precision    recall  f1-score   support

           0       0.78      0.60      0.68       834
           1       0.89      0.95      0.92      2943

   micro avg       0.87      0.87      0.87      3777
   macro avg       0.84      0.78      0.80      3777
weighted avg       0.87      0.87      0.87      3777



### Experiment 4 - use significant columns only to build the model 


In [626]:
data = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")

In [628]:
data.drop(['REP_VOTES', 'DEM_VOTES', 'MARGIN_VICTORY', 'WINNING_CANDIDATE', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)
data.drop(['AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE', 'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE' , 'TOT_FEMALE', 'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19', 'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39', 'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59', 'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60', 'TOT_MALE_Above60', 'TOT_FEMALE_Above60'] , axis=1, inplace=True)


In [600]:
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year, True)

KeyError: 'YEAR'

In [262]:
X_train

,YEAR_FIPS,YEAR,STATE_FIPS,COUNTY_FIPS,COUNTY_TOTALVOTES,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%
14861,200847107,2008,47,107,18789.0,1,1,8.7,36125,0.003833,0.003872,0.020149,0.021190,0.011943,0.014255,0.001946,0.001811,0.000096,0.000116,0.480891,0.451206,0.514679,0.485321,0.259612,0.237344,0.287641,0.215403
13109,200818159,2008,18,159,7827.0,1,1,7.2,68924,0.001845,0.001599,0.000800,0.001107,0.009042,0.009657,0.001169,0.000984,0.000062,0.000308,0.502061,0.483053,0.509504,0.490496,0.262718,0.217691,0.296365,0.223227
553,20166091,2016,6,91,1830.0,0,-1,7.6,74388,0.003356,0.002013,0.000336,0.002685,0.047315,0.050336,0.008054,0.011409,0.000336,0.000671,0.475839,0.473826,0.496980,0.503020,0.178188,0.155369,0.282886,0.383557
14883,200847151,2008,47,151,6783.0,1,1,11.5,0,0.000992,0.000857,0.000631,0.000631,0.002886,0.002976,0.001578,0.001217,0.000000,0.000045,0.501060,0.484963,0.507958,0.492042,0.281618,0.254836,0.279228,0.184318
1906,201229205,2012,29,205,3232.0,1,1,6.0,0,0.001571,0.000628,0.002199,0.002670,0.005655,0.005184,0.001257,0.000471,0.000000,0.000000,0.500157,0.483506,0.509268,0.490732,0.271913,0.204681,0.270657,0.252749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11633,200446077,2004,46,77,3014.0,1,0,4.0,23462,0.001656,0.002208,0.000368,0.000184,0.004416,0.004968,0.001656,0.002760,0.000000,0.000000,0.496964,0.489052,0.503956,0.496044,0.258878,0.167249,0.289052,0.284821
1344,201221059,2012,21,59,42087.0,1,1,7.0,0,0.003919,0.003361,0.023277,0.025335,0.011406,0.014818,0.000776,0.000900,0.000569,0.000703,0.477892,0.446963,0.514611,0.485389,0.269549,0.245755,0.279900,0.204796
12815,200813193,2008,13,193,4984.0,1,-1,8.4,0,0.005277,0.006527,0.297459,0.306485,0.011387,0.020275,0.001319,0.001944,0.000069,0.000972,0.173170,0.199486,0.480767,0.519233,0.263852,0.265102,0.296695,0.174351
7293,200025007,2000,25,7,8856.0,0,-1,2.7,28208,0.002933,0.002333,0.012866,0.012799,0.004933,0.005533,0.008933,0.009399,0.000533,0.000600,0.477835,0.454970,0.511366,0.488634,0.244584,0.242317,0.332511,0.180588


In [263]:
modelx = LogisticRegression()

In [264]:
#Recursive Feature Elimination
from sklearn.feature_selection import RFE


rfe = RFE(modelx, 20)
rfe = rfe.fit(X_train, y_train.values.ravel())

values = rfe.support_.tolist()

cols = []

for val, column in zip(values, X_train.columns):
    if val == True:
        cols.append(column)
        
cols

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: Futur

['YEAR_FIPS',
 'YEAR',
 'STATE_FIPS',
 'COUNTY_FIPS',
 'COUNTY_TOTALVOTES',
 'HOUSE_WINNING_BINARY',
 'SENATE_WINNING_BINARY',
 'UNEMPLOYMENT_RATE',
 'AVG_WAGE_SALARY',
 'BA_FEMALE%',
 'BA_MALE%',
 'H_FEMALE%',
 'IA_FEMALE%',
 'WA_FEMALE%',
 'WA_MALE%',
 'TOT_FEMALE%',
 'TOT_MALE%',
 'TOT_POP_LESS19%',
 'TOT_POP_40to59%',
 'TOT_POP_Above60%']

In [282]:
#remove YEAR_FIPS
cols.remove('YEAR_FIPS')

In [284]:
#split by: train=1; significant columns
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year, True, cols)

In [285]:
X_train

,YEAR,STATE_FIPS,COUNTY_FIPS,COUNTY_TOTALVOTES,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,BA_FEMALE%,BA_MALE%,H_FEMALE%,IA_FEMALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_40to59%,TOT_POP_Above60%
14861,2008,47,107,18789.0,1,1,8.7,36125,0.020149,0.021190,0.011943,0.001946,0.480891,0.451206,0.514679,0.485321,0.259612,0.287641,0.215403
13109,2008,18,159,7827.0,1,1,7.2,68924,0.000800,0.001107,0.009042,0.001169,0.502061,0.483053,0.509504,0.490496,0.262718,0.296365,0.223227
553,2016,6,91,1830.0,0,-1,7.6,74388,0.000336,0.002685,0.047315,0.008054,0.475839,0.473826,0.496980,0.503020,0.178188,0.282886,0.383557
14883,2008,47,151,6783.0,1,1,11.5,0,0.000631,0.000631,0.002886,0.001578,0.501060,0.484963,0.507958,0.492042,0.281618,0.279228,0.184318
1906,2012,29,205,3232.0,1,1,6.0,0,0.002199,0.002670,0.005655,0.001257,0.500157,0.483506,0.509268,0.490732,0.271913,0.270657,0.252749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11633,2004,46,77,3014.0,1,0,4.0,23462,0.000368,0.000184,0.004416,0.001656,0.496964,0.489052,0.503956,0.496044,0.258878,0.289052,0.284821
1344,2012,21,59,42087.0,1,1,7.0,0,0.023277,0.025335,0.011406,0.000776,0.477892,0.446963,0.514611,0.485389,0.269549,0.279900,0.204796
12815,2008,13,193,4984.0,1,-1,8.4,0,0.297459,0.306485,0.011387,0.001319,0.173170,0.199486,0.480767,0.519233,0.263852,0.296695,0.174351
7293,2000,25,7,8856.0,0,-1,2.7,28208,0.012866,0.012799,0.004933,0.008933,0.477835,0.454970,0.511366,0.488634,0.244584,0.332511,0.180588


In [286]:
#model = LogisticRegression(C=0.0001, solver='lbfgs', dual=False, max_iter=5000)

In [287]:
model4 = LogisticRegression()

In [288]:
model4.fit(X_test, y_test)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [289]:
preditions = model4.predict(X_test)

In [290]:
print("Experiment#4: {:.3f}%".format(accuracy_score(y_test, predictions) * 100))

Experiment#4: 80.636%


In [291]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.69      0.17      0.27      1005
           1       0.81      0.98      0.89      3715

   micro avg       0.81      0.81      0.81      4720
   macro avg       0.75      0.57      0.58      4720
weighted avg       0.79      0.81      0.76      4720



In [292]:
confusion_matrix(y_test, predictions)

array([[ 167,  838],
       [  76, 3639]])

In [293]:
rfc4 = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

rfc_pred4= rfc4.predict(X_test)

print("Experiment#rfc4: {:.3f}%".format(accuracy_score(y_test, rfc_pred4) * 100))

print(classification_report(y_test, rfc_pred4))

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


Experiment#rfc4: 89.004%
              precision    recall  f1-score   support

           0       0.80      0.64      0.71      1005
           1       0.91      0.96      0.93      3715

   micro avg       0.89      0.89      0.89      4720
   macro avg       0.85      0.80      0.82      4720
weighted avg       0.89      0.89      0.89      4720



In [624]:
cols

['YEAR',
 'STATE_FIPS',
 'COUNTY_FIPS',
 'COUNTY_TOTALVOTES',
 'HOUSE_WINNING_BINARY',
 'SENATE_WINNING_BINARY',
 'UNEMPLOYMENT_RATE',
 'AVG_WAGE_SALARY',
 'BA_FEMALE%',
 'BA_MALE%',
 'H_FEMALE%',
 'IA_FEMALE%',
 'WA_FEMALE%',
 'WA_MALE%',
 'TOT_FEMALE%',
 'TOT_MALE%',
 'TOT_POP_LESS19%',
 'TOT_POP_40to59%',
 'TOT_POP_Above60%']

In [630]:
data

,YEAR_FIPS,YEAR,STATE_FIPS,COUNTY_FIPS,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%
0,20121001,2012,1,1,23932.0,1,1,1,6.900000,33766,0.005406,0.003671,0.094094,0.083848,0.011232,0.012657,0.002393,0.002173,0.000310,0.000511,0.402747,0.390054,0.512912,0.487088,0.293056,0.252858,0.282427,0.171659
1,20121003,2012,1,3,85338.0,1,1,1,7.500000,64282,0.004549,0.003009,0.048675,0.045988,0.019737,0.024302,0.003659,0.003604,0.000300,0.000399,0.446461,0.429892,0.510581,0.489419,0.253453,0.230183,0.282293,0.234071
2,20121005,2012,1,5,11509.0,0,1,1,11.500000,64168,0.002159,0.002159,0.223600,0.248299,0.021917,0.027333,0.002671,0.003220,0.000622,0.001061,0.237175,0.271277,0.469301,0.530699,0.242225,0.270801,0.278741,0.208233
3,20121007,2012,1,7,8420.0,1,1,1,8.500000,69624,0.000437,0.000525,0.086219,0.134182,0.004809,0.013116,0.001399,0.001574,0.000219,0.001137,0.371240,0.394981,0.463799,0.536201,0.248820,0.283097,0.280955,0.187128
4,20121009,2012,1,9,24006.0,1,1,1,6.900000,60178,0.001429,0.000837,0.006798,0.007390,0.036271,0.045004,0.003015,0.003085,0.000436,0.000627,0.487947,0.477838,0.505255,0.494745,0.271033,0.241682,0.276594,0.210691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15728,201246113,2012,46,113,3145.0,0,1,1,5.706686,0,0.000080,0.000241,0.000080,0.000722,0.006578,0.007621,0.473127,0.470399,0.000160,0.000321,0.024065,0.022862,0.501043,0.498957,0.490454,0.267367,0.169501,0.072678
15729,201646113,2016,46,113,2905.0,0,1,1,5.706686,0,0.000077,0.000308,0.000077,0.000462,0.009159,0.008928,0.477180,0.469561,0.000077,0.000308,0.022397,0.020857,0.503810,0.496190,0.475872,0.264296,0.182098,0.077734
15730,2000102,2000,46,102,30000.0,1,1,1,14.200000,25000,0.000293,0.000733,0.000220,0.000073,0.011440,0.011734,0.486213,0.472646,0.000000,0.000220,0.015620,0.015474,0.506967,0.493033,0.434878,0.272954,0.203652,0.088516
15731,2004102,2004,46,102,30000.0,1,1,1,14.200000,25000,0.000293,0.000733,0.000220,0.000073,0.011440,0.011734,0.486213,0.472646,0.000000,0.000220,0.015620,0.015474,0.506967,0.493033,0.434878,0.272954,0.203652,0.088516


In [642]:
###### with new train split
cols.remove('YEAR')
X = data.drop('WINNING_PARTY_BINARY', axis=1)
y = data[['WINNING_PARTY_BINARY', 'YEAR']].values

X_train, X_test, y_train, y_test = train_test_split_by_year(X, y, year, cols)

ValueError: list.remove(x): x not in list

In [634]:
rfc4 = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

rfc_pred4= rfc4.predict(X_test)

print("Experiment#rfc4: {:.3f}%".format(accuracy_score(y_test, rfc_pred4) * 100))

print(classification_report(y_test, rfc_pred4))

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


Experiment#rfc4: 88.642%
              precision    recall  f1-score   support

           0       0.82      0.64      0.72       858
           1       0.90      0.96      0.93      2919

   micro avg       0.89      0.89      0.89      3777
   macro avg       0.86      0.80      0.82      3777
weighted avg       0.88      0.89      0.88      3777



# Predict Target Year -- Step 2

In [635]:
target_year_data = pd.read_csv('../../data/F20_phase1_0717_target.csv', encoding = "ISO-8859-1")

In [636]:
target_year_data.columns

Index(['Unnamed: 0', 'YEAR_FIPS', 'STATE_FIPS', 'COUNTY_FIPS',
       'HOUSE_WINNING_BINARY', 'SENATE_WINNING_BINARY', 'UNEMPLOYMENT_RATE',
       'AVG_WAGE_SALARY', 'AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE',
       'H_FEMALE', 'H_MALE', 'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE',
       'TOT_FEMALE', 'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE',
       'TOT_POP_LESS19', 'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19',
       'TOT_POP_20to39', 'TOT_MALE_20to39', 'TOT_FEMALE_20to39',
       'TOT_POP_40to59', 'TOT_MALE_40to59', 'TOT_FEMALE_40to59',
       'TOT_POP_Above60', 'TOT_MALE_Above60', 'TOT_FEMALE_Above60',
       'AA_FEMALE%', 'AA_MALE%', 'BA_FEMALE%', 'BA_MALE%', 'H_FEMALE%',
       'H_MALE%', 'IA_FEMALE%', 'IA_MALE%', 'NA_FEMALE%', 'NA_MALE%',
       'WA_FEMALE%', 'WA_MALE%', 'TOT_FEMALE%', 'TOT_MALE%', 'TOT_POP_LESS19%',
       'TOT_POP_20to39%', 'TOT_POP_40to59%', 'TOT_POP_Above60%',
       'COUNTY_TOTALVOTES'],
      dtype='object')

In [637]:
target_year_data

,Unnamed: 0,YEAR_FIPS,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES
0,319,20161001,1,1,1,1,5.100000,36564,337,237,5298,4807,697,733,153,125,18,32,28092,26784,54876,21841,21085,14950,7687,7263,13760,6775,6985,15715,7623,8092,10451,4699,5752,0.006141,0.004319,0.096545,0.087597,0.012701,0.013357,0.002788,0.002278,0.000328,0.000583,0.398006,0.384230,0.511918,0.488082,0.272432,0.250747,0.286373,0.190448,22086
1,320,20161003,1,3,1,1,5.300000,69804,1154,751,9602,9142,4110,4566,749,758,55,59,102034,97155,199189,88807,84896,48612,24596,24016,45459,22648,22811,55015,26533,28482,50103,23378,26725,0.005793,0.003770,0.048205,0.045896,0.020634,0.022923,0.003760,0.003805,0.000276,0.000296,0.445843,0.426208,0.512247,0.487753,0.244050,0.228220,0.276195,0.251535,97879
2,321,20161005,1,5,1,1,8.300000,70238,55,60,6041,6782,497,641,66,92,15,27,12514,14244,26758,6236,7139,6200,3109,3091,7215,4444,2771,7207,3947,3260,6136,2744,3392,0.002055,0.002242,0.225764,0.253457,0.018574,0.023955,0.002467,0.003438,0.000561,0.001009,0.233052,0.266799,0.467673,0.532327,0.231706,0.269639,0.269340,0.229315,11533
3,322,20161007,1,7,1,1,6.400000,79056,23,15,1817,3071,196,290,38,44,4,23,10402,12139,22541,8414,8889,5179,2766,2413,6237,3716,2521,6485,3513,2972,4640,2144,2496,0.001020,0.000665,0.080609,0.136241,0.008695,0.012865,0.001686,0.001952,0.000177,0.001020,0.373275,0.394348,0.461470,0.538530,0.229759,0.276696,0.287698,0.205847,5493
4,323,20161009,1,9,1,1,5.400000,66760,93,73,418,485,2301,2718,164,183,24,39,29152,28369,57521,28022,27283,14846,7504,7342,13550,6830,6720,15813,7950,7863,13312,6085,7227,0.001617,0.001269,0.007267,0.008432,0.040003,0.047252,0.002851,0.003181,0.000417,0.000678,0.487161,0.474314,0.506806,0.493194,0.258097,0.235566,0.274908,0.231429,26353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141,6286,201656039,56,39,1,1,3.500000,0,156,116,107,129,1622,1797,105,101,12,14,10966,11811,22777,10433,11275,4639,2343,2296,7397,3913,3484,6491,3379,3112,4250,2176,2074,0.006849,0.005093,0.004698,0.005664,0.071212,0.078895,0.004610,0.004434,0.000527,0.000615,0.458050,0.495017,0.481451,0.518549,0.203670,0.324757,0.284980,0.186592,9814
3142,6287,201656041,56,41,1,1,5.800000,40941,58,34,64,79,905,994,134,129,19,23,10278,10549,20827,9852,10085,6699,3462,3237,5244,2620,2624,5314,2670,2644,3570,1797,1773,0.002785,0.001632,0.003073,0.003793,0.043453,0.047727,0.006434,0.006194,0.000912,0.001104,0.473040,0.484227,0.493494,0.506506,0.321650,0.251789,0.255150,0.171412,5305
3143,6288,201656043,56,43,1,1,4.800000,41058,33,24,20,24,556,597,75,73,4,5,4110,4165,8275,3893,3962,2192,1101,1091,1727,897,830,2128,1077,1051,2228,1090,1138,0.003988,0.002900,0.002417,0.002900,0.067190,0.072145,0.009063,0.008822,0.000483,0.000604,0.470453,0.478792,0.496677,0.503323,0.264894,0.208701,0.257160,0.269245,4313
3144,6289,201656045,56,45,1,1,5.000000,38311,25,21,18,27,118,166,49,67,0,2,3370,3768,7138,3209,3572,1693,865,828,1675,956,719,1962,1058,904,1808,889,919,0.003502,0.002942,0.002522,0.003783,0.016531,0.023256,0.006865,0.009386,0.000000,0.000280,0.449566,0.500420,0.472121,0.527879,0.237181,0.234660,0.274867,0.253292,4171


In [638]:
data_2020.isna().sum()

YEAR_FIPS              0
YEAR                   0
STATE_FIPS             0
COUNTY_FIPS            0
STATE                  0
                    ... 
TOT_POP_LESS19%        0
TOT_POP_20to39%        0
TOT_POP_40to59%        0
TOT_POP_Above60%       0
MARGIN_VICTORY      3149
Length: 64, dtype: int64

In [639]:
## use with significant model only..
## for now drop AVG_WAGE_SALARY till data is fixed. 
data_significant = target_year_data[[
 'STATE_FIPS',
 'COUNTY_FIPS',
 'COUNTY_TOTALVOTES',
 'HOUSE_WINNING_BINARY',
 'SENATE_WINNING_BINARY',
 'UNEMPLOYMENT_RATE',
 'AVG_WAGE_SALARY',
 'BA_FEMALE%',
 'BA_MALE%',
 'H_FEMALE%',
 'IA_FEMALE%',
 'WA_FEMALE%',
 'WA_MALE%',
 'TOT_FEMALE%',
 'TOT_MALE%',
 'TOT_POP_LESS19%',
 'TOT_POP_40to59%',
 'TOT_POP_Above60%']]

In [645]:
data_significant.isna().sum()

STATE_FIPS               0
COUNTY_FIPS              0
COUNTY_TOTALVOTES        0
HOUSE_WINNING_BINARY     0
SENATE_WINNING_BINARY    0
UNEMPLOYMENT_RATE        0
AVG_WAGE_SALARY          0
BA_FEMALE%               0
BA_MALE%                 0
H_FEMALE%                0
IA_FEMALE%               0
WA_FEMALE%               0
WA_MALE%                 0
TOT_FEMALE%              0
TOT_MALE%                0
TOT_POP_LESS19%          0
TOT_POP_40to59%          0
TOT_POP_Above60%         0
dtype: int64

In [646]:
X_train.isna().sum()

STATE_FIPS               0
COUNTY_FIPS              0
COUNTY_TOTALVOTES        0
HOUSE_WINNING_BINARY     0
SENATE_WINNING_BINARY    0
UNEMPLOYMENT_RATE        0
AVG_WAGE_SALARY          0
BA_FEMALE%               0
BA_MALE%                 0
H_FEMALE%                0
IA_FEMALE%               0
WA_FEMALE%               0
WA_MALE%                 0
TOT_FEMALE%              0
TOT_MALE%                0
TOT_POP_LESS19%          0
TOT_POP_40to59%          0
TOT_POP_Above60%         0
dtype: int64

In [647]:
y_pred = rfc4.predict(data_significant)

In [648]:
target_year_data['WINNING_PARTY_BINARY'] = y_pred

In [649]:
target_year_data

,Unnamed: 0,YEAR_FIPS,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY
0,319,20161001,1,1,1,1,5.100000,36564,337,237,5298,4807,697,733,153,125,18,32,28092,26784,54876,21841,21085,14950,7687,7263,13760,6775,6985,15715,7623,8092,10451,4699,5752,0.006141,0.004319,0.096545,0.087597,0.012701,0.013357,0.002788,0.002278,0.000328,0.000583,0.398006,0.384230,0.511918,0.488082,0.272432,0.250747,0.286373,0.190448,22086,1
1,320,20161003,1,3,1,1,5.300000,69804,1154,751,9602,9142,4110,4566,749,758,55,59,102034,97155,199189,88807,84896,48612,24596,24016,45459,22648,22811,55015,26533,28482,50103,23378,26725,0.005793,0.003770,0.048205,0.045896,0.020634,0.022923,0.003760,0.003805,0.000276,0.000296,0.445843,0.426208,0.512247,0.487753,0.244050,0.228220,0.276195,0.251535,97879,1
2,321,20161005,1,5,1,1,8.300000,70238,55,60,6041,6782,497,641,66,92,15,27,12514,14244,26758,6236,7139,6200,3109,3091,7215,4444,2771,7207,3947,3260,6136,2744,3392,0.002055,0.002242,0.225764,0.253457,0.018574,0.023955,0.002467,0.003438,0.000561,0.001009,0.233052,0.266799,0.467673,0.532327,0.231706,0.269639,0.269340,0.229315,11533,0
3,322,20161007,1,7,1,1,6.400000,79056,23,15,1817,3071,196,290,38,44,4,23,10402,12139,22541,8414,8889,5179,2766,2413,6237,3716,2521,6485,3513,2972,4640,2144,2496,0.001020,0.000665,0.080609,0.136241,0.008695,0.012865,0.001686,0.001952,0.000177,0.001020,0.373275,0.394348,0.461470,0.538530,0.229759,0.276696,0.287698,0.205847,5493,1
4,323,20161009,1,9,1,1,5.400000,66760,93,73,418,485,2301,2718,164,183,24,39,29152,28369,57521,28022,27283,14846,7504,7342,13550,6830,6720,15813,7950,7863,13312,6085,7227,0.001617,0.001269,0.007267,0.008432,0.040003,0.047252,0.002851,0.003181,0.000417,0.000678,0.487161,0.474314,0.506806,0.493194,0.258097,0.235566,0.274908,0.231429,26353,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141,6286,201656039,56,39,1,1,3.500000,0,156,116,107,129,1622,1797,105,101,12,14,10966,11811,22777,10433,11275,4639,2343,2296,7397,3913,3484,6491,3379,3112,4250,2176,2074,0.006849,0.005093,0.004698,0.005664,0.071212,0.078895,0.004610,0.004434,0.000527,0.000615,0.458050,0.495017,0.481451,0.518549,0.203670,0.324757,0.284980,0.186592,9814,1
3142,6287,201656041,56,41,1,1,5.800000,40941,58,34,64,79,905,994,134,129,19,23,10278,10549,20827,9852,10085,6699,3462,3237,5244,2620,2624,5314,2670,2644,3570,1797,1773,0.002785,0.001632,0.003073,0.003793,0.043453,0.047727,0.006434,0.006194,0.000912,0.001104,0.473040,0.484227,0.493494,0.506506,0.321650,0.251789,0.255150,0.171412,5305,1
3143,6288,201656043,56,43,1,1,4.800000,41058,33,24,20,24,556,597,75,73,4,5,4110,4165,8275,3893,3962,2192,1101,1091,1727,897,830,2128,1077,1051,2228,1090,1138,0.003988,0.002900,0.002417,0.002900,0.067190,0.072145,0.009063,0.008822,0.000483,0.000604,0.470453,0.478792,0.496677,0.503323,0.264894,0.208701,0.257160,0.269245,4313,1
3144,6289,201656045,56,45,1,1,5.000000,38311,25,21,18,27,118,166,49,67,0,2,3370,3768,7138,3209,3572,1693,865,828,1675,956,719,1962,1058,904,1808,889,919,0.003502,0.002942,0.002522,0.003783,0.016531,0.023256,0.006865,0.009386,0.000000,0.000280,0.449566,0.500420,0.472121,0.527879,0.237181,0.234660,0.274867,0.253292,4171,1


In [650]:
#save to csv
target_year_data.to_csv("../../data/F20_phase2_0717_target.csv")

In [651]:
target_year_data.loc[target_year_data['WINNING_PARTY_BINARY'] == 1]

,Unnamed: 0,YEAR_FIPS,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY
0,319,20161001,1,1,1,1,5.1,36564,337,237,5298,4807,697,733,153,125,18,32,28092,26784,54876,21841,21085,14950,7687,7263,13760,6775,6985,15715,7623,8092,10451,4699,5752,0.006141,0.004319,0.096545,0.087597,0.012701,0.013357,0.002788,0.002278,0.000328,0.000583,0.398006,0.384230,0.511918,0.488082,0.272432,0.250747,0.286373,0.190448,22086,1
1,320,20161003,1,3,1,1,5.3,69804,1154,751,9602,9142,4110,4566,749,758,55,59,102034,97155,199189,88807,84896,48612,24596,24016,45459,22648,22811,55015,26533,28482,50103,23378,26725,0.005793,0.003770,0.048205,0.045896,0.020634,0.022923,0.003760,0.003805,0.000276,0.000296,0.445843,0.426208,0.512247,0.487753,0.244050,0.228220,0.276195,0.251535,97879,1
3,322,20161007,1,7,1,1,6.4,79056,23,15,1817,3071,196,290,38,44,4,23,10402,12139,22541,8414,8889,5179,2766,2413,6237,3716,2521,6485,3513,2972,4640,2144,2496,0.001020,0.000665,0.080609,0.136241,0.008695,0.012865,0.001686,0.001952,0.000177,0.001020,0.373275,0.394348,0.461470,0.538530,0.229759,0.276696,0.287698,0.205847,5493,1
4,323,20161009,1,9,1,1,5.4,66760,93,73,418,485,2301,2718,164,183,24,39,29152,28369,57521,28022,27283,14846,7504,7342,13550,6830,6720,15813,7950,7863,13312,6085,7227,0.001617,0.001269,0.007267,0.008432,0.040003,0.047252,0.002851,0.003181,0.000417,0.000678,0.487161,0.474314,0.506806,0.493194,0.258097,0.235566,0.274908,0.231429,26353,1
6,325,20161013,1,13,1,1,6.9,266456,106,126,4968,4039,115,134,31,34,5,5,10894,9449,20343,5690,5156,5145,2670,2475,4873,2229,2644,5221,2395,2826,5104,2155,2949,0.005211,0.006194,0.244212,0.198545,0.005653,0.006587,0.001524,0.001671,0.000246,0.000246,0.279703,0.253453,0.535516,0.464484,0.252913,0.239542,0.256648,0.250897,7627,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,6285,201656037,56,37,1,1,6.1,57278,207,151,261,348,3313,3802,279,315,38,42,21665,23292,44957,20495,22009,13334,6874,6460,13166,6885,6281,11556,6075,5481,6901,3458,3443,0.004604,0.003359,0.005806,0.007741,0.073693,0.084570,0.006206,0.007007,0.000845,0.000934,0.455880,0.489557,0.481905,0.518095,0.296595,0.292858,0.257046,0.153502,14783,1
3141,6286,201656039,56,39,1,1,3.5,0,156,116,107,129,1622,1797,105,101,12,14,10966,11811,22777,10433,11275,4639,2343,2296,7397,3913,3484,6491,3379,3112,4250,2176,2074,0.006849,0.005093,0.004698,0.005664,0.071212,0.078895,0.004610,0.004434,0.000527,0.000615,0.458050,0.495017,0.481451,0.518549,0.203670,0.324757,0.284980,0.186592,9814,1
3142,6287,201656041,56,41,1,1,5.8,40941,58,34,64,79,905,994,134,129,19,23,10278,10549,20827,9852,10085,6699,3462,3237,5244,2620,2624,5314,2670,2644,3570,1797,1773,0.002785,0.001632,0.003073,0.003793,0.043453,0.047727,0.006434,0.006194,0.000912,0.001104,0.473040,0.484227,0.493494,0.506506,0.321650,0.251789,0.255150,0.171412,5305,1
3143,6288,201656043,56,43,1,1,4.8,41058,33,24,20,24,556,597,75,73,4,5,4110,4165,8275,3893,3962,2192,1101,1091,1727,897,830,2128,1077,1051,2228,1090,1138,0.003988,0.002900,0.002417,0.002900,0.067190,0.072145,0.009063,0.008822,0.000483,0.000604,0.470453,0.478792,0.496677,0.503323,0.264894,0.208701,0.257160,0.269245,4313,1


# 2020 Step 3: D vs R Votes

In [ ]:
###-------- take 2 -------##

In [652]:
votes_historcail_df = pd.read_csv(combined_data_file, encoding = "ISO-8859-1")
#votes_df = pd.read_csv("../../data/F20_predictions_jun28.csv")
votes_df = pd.read_csv("../../data/F20_phase2_0717_target.csv")

In [653]:
votes_historcail_df_copy = votes_historcail_df[['YEAR', 'STATE_FIPS', 'COUNTY_FIPS', 'COUNTY_TOTALVOTES', 'WINNING_PARTY_BINARY', 'REP_VOTES']]

In [654]:
votes_historcail_df_copy['REP_VOTES%'] = votes_historcail_df_copy['REP_VOTES'] / votes_historcail_df_copy['COUNTY_TOTALVOTES']
votes_historcail_df_copy.drop('REP_VOTES', axis=1, inplace=True)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [655]:
#votes_historcail_df_copy.drop('WINNING_PARTY_BINARY', axis=1, inplace=True)

In [656]:
votes_historcail_df_copy.sample(20)

,YEAR,STATE_FIPS,COUNTY_FIPS,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES%
15589,2008,56,29,15049.0,1,0.720247
5075,2016,37,75,4168.0,1,0.787668
13511,2008,22,83,9181.0,1,0.626402
7426,2000,27,79,12331.0,1,0.497770
1421,2012,21,213,7093.0,1,0.613986
8308,2000,41,25,3734.0,1,0.749598
4560,2016,28,23,7791.0,1,0.659351
11322,2004,39,121,6540.0,1,0.587309
9243,2000,55,65,7263.0,0,0.459314
4580,2016,28,63,3860.0,0,0.126943


In [657]:
#votes_historcail_df_copy['COUNTY_TOTALVOTES'] = votes_historcail_df_copy['COUNTY_TOTALVOTES'].astype(np.int64)
#votes_historcail_df_copy['REP_VOTES'] = votes_historcail_df_copy['REP_VOTES'].astype(np.int64)

In [658]:
votes_historcail_df_copy

,YEAR,STATE_FIPS,COUNTY_FIPS,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES%
0,2012,1,1,23932.0,1,0.726183
1,2012,1,3,85338.0,1,0.773583
2,2012,1,5,11509.0,0,0.482231
3,2012,1,7,8420.0,1,0.728266
4,2012,1,9,24006.0,1,0.864659
...,...,...,...,...,...,...
15728,2012,46,113,3145.0,0,0.059777
15729,2016,46,113,2905.0,0,0.082960
15730,2000,46,102,30000.0,1,0.600000
15731,2004,46,102,30000.0,1,0.600000


In [659]:
X = votes_historcail_df_copy.drop(['REP_VOTES%'], axis=1)
y = votes_historcail_df_copy[['REP_VOTES%', 'YEAR']].values

In [660]:
X_train, X_test, y_train, y_test = train_test_split_by_year(X, y, year)

In [661]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [662]:
y_pred = linear_model.predict(X_test)

In [663]:
print("Accuracy score={:.2f}".format(linear_model.score(X_test, y_test) * 100))

Accuracy score=53.92


In [664]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.07469144635021506
Mean Squared Error: 0.008749809537991956
Root Mean Squared Error: 0.09354041660155227


In [665]:
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df.head(25)

,Actual,Predicted
0,0.647299,0.612807
1,0.684827,0.642516
2,0.635702,0.631509
3,0.509405,0.642306
4,0.452373,0.403203
5,0.182942,0.350537
6,0.557234,0.638693
7,0.500709,0.639494
8,0.658839,0.642362
9,0.797960,0.645248


In [666]:
X_train.columns

Index(['STATE_FIPS', 'COUNTY_FIPS', 'COUNTY_TOTALVOTES',
       'WINNING_PARTY_BINARY'],
      dtype='object')

In [667]:
votes_df_copy = votes_df[['STATE_FIPS', 'COUNTY_FIPS', 'COUNTY_TOTALVOTES', 'WINNING_PARTY_BINARY']]

In [668]:
y_pred = linear_model.predict(votes_df_copy)

In [671]:
target_year_data['REP_VOTES%'] = y_pred
#data_2020['REP_VOTES'] = data_2020['REP_VOTES'].astype(np.int64)

In [672]:
target_year_data

,Unnamed: 0,YEAR_FIPS,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES%
0,319,20161001,1,1,1,1,5.100000,36564,337,237,5298,4807,697,733,153,125,18,32,28092,26784,54876,21841,21085,14950,7687,7263,13760,6775,6985,15715,7623,8092,10451,4699,5752,0.006141,0.004319,0.096545,0.087597,0.012701,0.013357,0.002788,0.002278,0.000328,0.000583,0.398006,0.384230,0.511918,0.488082,0.272432,0.250747,0.286373,0.190448,22086,1,0.620125
1,320,20161003,1,3,1,1,5.300000,69804,1154,751,9602,9142,4110,4566,749,758,55,59,102034,97155,199189,88807,84896,48612,24596,24016,45459,22648,22811,55015,26533,28482,50103,23378,26725,0.005793,0.003770,0.048205,0.045896,0.020634,0.022923,0.003760,0.003805,0.000276,0.000296,0.445843,0.426208,0.512247,0.487753,0.244050,0.228220,0.276195,0.251535,97879,1,0.612631
2,321,20161005,1,5,1,1,8.300000,70238,55,60,6041,6782,497,641,66,92,15,27,12514,14244,26758,6236,7139,6200,3109,3091,7215,4444,2771,7207,3947,3260,6136,2744,3392,0.002055,0.002242,0.225764,0.253457,0.018574,0.023955,0.002467,0.003438,0.000561,0.001009,0.233052,0.266799,0.467673,0.532327,0.231706,0.269639,0.269340,0.229315,11533,0,0.390985
3,322,20161007,1,7,1,1,6.400000,79056,23,15,1817,3071,196,290,38,44,4,23,10402,12139,22541,8414,8889,5179,2766,2413,6237,3716,2521,6485,3513,2972,4640,2144,2496,0.001020,0.000665,0.080609,0.136241,0.008695,0.012865,0.001686,0.001952,0.000177,0.001020,0.373275,0.394348,0.461470,0.538530,0.229759,0.276696,0.287698,0.205847,5493,1,0.622246
4,323,20161009,1,9,1,1,5.400000,66760,93,73,418,485,2301,2718,164,183,24,39,29152,28369,57521,28022,27283,14846,7504,7342,13550,6830,6720,15813,7950,7863,13312,6085,7227,0.001617,0.001269,0.007267,0.008432,0.040003,0.047252,0.002851,0.003181,0.000417,0.000678,0.487161,0.474314,0.506806,0.493194,0.258097,0.235566,0.274908,0.231429,26353,1,0.620292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141,6286,201656039,56,39,1,1,3.500000,0,156,116,107,129,1622,1797,105,101,12,14,10966,11811,22777,10433,11275,4639,2343,2296,7397,3913,3484,6491,3379,3112,4250,2176,2074,0.006849,0.005093,0.004698,0.005664,0.071212,0.078895,0.004610,0.004434,0.000527,0.000615,0.458050,0.495017,0.481451,0.518549,0.203670,0.324757,0.284980,0.186592,9814,1,0.644190
3142,6287,201656041,56,41,1,1,5.800000,40941,58,34,64,79,905,994,134,129,19,23,10278,10549,20827,9852,10085,6699,3462,3237,5244,2620,2624,5314,2670,2644,3570,1797,1773,0.002785,0.001632,0.003073,0.003793,0.043453,0.047727,0.006434,0.006194,0.000912,0.001104,0.473040,0.484227,0.493494,0.506506,0.321650,0.251789,0.255150,0.171412,5305,1,0.644794
3143,6288,201656043,56,43,1,1,4.800000,41058,33,24,20,24,556,597,75,73,4,5,4110,4165,8275,3893,3962,2192,1101,1091,1727,897,830,2128,1077,1051,2228,1090,1138,0.003988,0.002900,0.002417,0.002900,0.067190,0.072145,0.009063,0.008822,0.000483,0.000604,0.470453,0.478792,0.496677,0.503323,0.264894,0.208701,0.257160,0.269245,4313,1,0.645043
3144,6289,201656045,56,45,1,1,5.000000,38311,25,21,18,27,118,166,49,67,0,2,3370,3768,7138,3209,3572,1693,865,828,1675,956,719,1962,1058,904,1808,889,919,0.003502,0.002942,0.002522,0.003783,0.016531,0.023256,0.006865,0.009386,0.000000,0.000280,0.449566,0.500420,0.472121,0.527879,0.237181

In [673]:
#info D total votes. 
target_year_data['DEM_VOTES%'] = 1 - target_year_data['REP_VOTES%']

In [674]:
target_year_data

,Unnamed: 0,YEAR_FIPS,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES%,DEM_VOTES%
0,319,20161001,1,1,1,1,5.100000,36564,337,237,5298,4807,697,733,153,125,18,32,28092,26784,54876,21841,21085,14950,7687,7263,13760,6775,6985,15715,7623,8092,10451,4699,5752,0.006141,0.004319,0.096545,0.087597,0.012701,0.013357,0.002788,0.002278,0.000328,0.000583,0.398006,0.384230,0.511918,0.488082,0.272432,0.250747,0.286373,0.190448,22086,1,0.620125,0.379875
1,320,20161003,1,3,1,1,5.300000,69804,1154,751,9602,9142,4110,4566,749,758,55,59,102034,97155,199189,88807,84896,48612,24596,24016,45459,22648,22811,55015,26533,28482,50103,23378,26725,0.005793,0.003770,0.048205,0.045896,0.020634,0.022923,0.003760,0.003805,0.000276,0.000296,0.445843,0.426208,0.512247,0.487753,0.244050,0.228220,0.276195,0.251535,97879,1,0.612631,0.387369
2,321,20161005,1,5,1,1,8.300000,70238,55,60,6041,6782,497,641,66,92,15,27,12514,14244,26758,6236,7139,6200,3109,3091,7215,4444,2771,7207,3947,3260,6136,2744,3392,0.002055,0.002242,0.225764,0.253457,0.018574,0.023955,0.002467,0.003438,0.000561,0.001009,0.233052,0.266799,0.467673,0.532327,0.231706,0.269639,0.269340,0.229315,11533,0,0.390985,0.609015
3,322,20161007,1,7,1,1,6.400000,79056,23,15,1817,3071,196,290,38,44,4,23,10402,12139,22541,8414,8889,5179,2766,2413,6237,3716,2521,6485,3513,2972,4640,2144,2496,0.001020,0.000665,0.080609,0.136241,0.008695,0.012865,0.001686,0.001952,0.000177,0.001020,0.373275,0.394348,0.461470,0.538530,0.229759,0.276696,0.287698,0.205847,5493,1,0.622246,0.377754
4,323,20161009,1,9,1,1,5.400000,66760,93,73,418,485,2301,2718,164,183,24,39,29152,28369,57521,28022,27283,14846,7504,7342,13550,6830,6720,15813,7950,7863,13312,6085,7227,0.001617,0.001269,0.007267,0.008432,0.040003,0.047252,0.002851,0.003181,0.000417,0.000678,0.487161,0.474314,0.506806,0.493194,0.258097,0.235566,0.274908,0.231429,26353,1,0.620292,0.379708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3141,6286,201656039,56,39,1,1,3.500000,0,156,116,107,129,1622,1797,105,101,12,14,10966,11811,22777,10433,11275,4639,2343,2296,7397,3913,3484,6491,3379,3112,4250,2176,2074,0.006849,0.005093,0.004698,0.005664,0.071212,0.078895,0.004610,0.004434,0.000527,0.000615,0.458050,0.495017,0.481451,0.518549,0.203670,0.324757,0.284980,0.186592,9814,1,0.644190,0.355810
3142,6287,201656041,56,41,1,1,5.800000,40941,58,34,64,79,905,994,134,129,19,23,10278,10549,20827,9852,10085,6699,3462,3237,5244,2620,2624,5314,2670,2644,3570,1797,1773,0.002785,0.001632,0.003073,0.003793,0.043453,0.047727,0.006434,0.006194,0.000912,0.001104,0.473040,0.484227,0.493494,0.506506,0.321650,0.251789,0.255150,0.171412,5305,1,0.644794,0.355206
3143,6288,201656043,56,43,1,1,4.800000,41058,33,24,20,24,556,597,75,73,4,5,4110,4165,8275,3893,3962,2192,1101,1091,1727,897,830,2128,1077,1051,2228,1090,1138,0.003988,0.002900,0.002417,0.002900,0.067190,0.072145,0.009063,0.008822,0.000483,0.000604,0.470453,0.478792,0.496677,0.503323,0.264894,0.208701,0.257160,0.269245,4313,1,0.645043,0.354957
3144,6289,201656045,56,45,1,1,5.000000,38311,25,21,18,27,118,166,49,67,0,2,3370,3768,7138,3209,3572,1693,865,828,1675,956,719,1962,1058,904,1808,889,919,0.003502,0.002942,0.002522,0.003783,0.016531,0.

In [675]:
#data_2020.loc[(data_2020['STATE_FIPS'] == 6) & (data_2020['WINNING_PARTY_BINARY'] == 0)]
target_year_data.loc[(target_year_data['STATE_FIPS'] == 6) ]

,Unnamed: 0,YEAR_FIPS,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES%,DEM_VOTES%
189,508,20166001,6,1,0,-1,4.3,71347,243563,223223,103211,91757,178801,183447,8672,9191,8112,7584,820526,788148,1608674,414415,415670,384662,195927,188735,486877,242200,244677,448057,220816,227241,289078,129205,159873,0.151406,0.138762,0.064159,0.057039,0.111148,0.114036,0.005391,0.005713,0.005043,0.004714,0.257613,0.258393,0.510064,0.489936,0.239117,0.302657,0.278526,0.179700,607664,0,0.332392,0.667608
190,509,20166003,6,3,0,-1,6.3,43438,11,7,1,1,45,54,137,124,0,0,529,553,1082,364,399,246,120,126,184,99,85,329,173,156,323,161,162,0.010166,0.006470,0.000924,0.000924,0.041590,0.049908,0.126617,0.114603,0.000000,0.000000,0.336414,0.368762,0.488909,0.511091,0.227357,0.170055,0.304067,0.298521,7262,1,0.623586,0.376414
191,510,20166005,6,5,0,-1,5.9,43648,276,246,90,678,1820,2886,336,464,43,44,17226,19512,36738,15876,17492,6280,3208,3072,7241,4264,2977,10581,5916,4665,12636,6124,6512,0.007513,0.006696,0.002450,0.018455,0.049540,0.078556,0.009146,0.012630,0.001170,0.001198,0.432141,0.476128,0.468888,0.531112,0.170940,0.197098,0.288012,0.343949,18969,1,0.622555,0.377445
192,511,20166007,6,7,0,-1,6.6,120924,5057,5066,1646,2170,16876,17503,2839,2739,301,309,112975,110595,223570,98312,95692,53375,27200,26175,65353,33802,31551,52232,25656,26576,52610,23937,28673,0.022619,0.022660,0.007362,0.009706,0.075484,0.078289,0.012698,0.012251,0.001346,0.001382,0.439737,0.428018,0.505323,0.494677,0.238740,0.292316,0.233627,0.235318,89168,0,0.385123,0.614877
193,512,20166009,6,9,0,-1,5.7,39007,360,281,151,261,2405,2602,404,411,47,48,22321,22352,44673,20581,20574,8800,4593,4207,7770,4059,3711,12534,6088,6446,15569,7612,7957,0.008059,0.006290,0.003380,0.005842,0.053836,0.058245,0.009043,0.009200,0.001052,0.001074,0.460703,0.460547,0.499653,0.500347,0.196987,0.173931,0.280572,0.348510,20618,1,0.622687,0.377313
194,513,20166011,6,11,0,-1,15.6,40410,202,175,113,142,5828,6452,269,305,60,60,10338,10860,21198,9463,9941,6551,3437,3114,5477,2848,2629,5176,2661,2515,3994,1914,2080,0.009529,0.008255,0.005331,0.006699,0.274932,0.304368,0.012690,0.014388,0.002830,0.002830,0.446410,0.468959,0.487688,0.512312,0.309039,0.258373,0.244174,0.188414,3198,1,0.624593,0.375407
195,514,20166013,6,13,0,-1,4.5,66525,95645,85042,56428,49586,138386,139323,5337,5622,3274,3056,568023,541335,1109358,378808,370358,287463,146786,140677,279081,138847,140234,319529,155661,163868,223285,100041,123244,0.086217,0.076659,0.050865,0.044698,0.124744,0.125589,0.004811,0.005068,0.002951,0.002755,0.341466,0.333849,0.512029,0.487971,0.259126,0.251570,0.288031,0.201274,478422,0,0.346320,0.653680
196,515,20166015,6,15,0,-1,7.5,38375,484,463,71,874,1768,3390,1232,1265,18,22,12330,14868,27198,9870,11546,6382,3279,3103,7350,4652,2698,7353,4015,3338,6113,2922,3191,0.017795,0.017023,0.002610,0.032135,0.065005,0.124642,0.045297,0.046511,0.000662,0.000809,0.362894,0.424517,0.453342,0.546658,0.234650,0.270240,0.270351,0.224759,11889,1,0.624015,0.375985
197,516,20166017,6,17,0,-1,5.1,48713,4073,3309,736,1001,11231,11510,1161,1265,201,179,91597,91564,183161,82226,82615,42400,21889,20511,37483,19323,18160,55486,27238,28248,47792,23114,24678,0.022237,0.018066,0.004018,0.005465,0.061318,0.062841,0.006339,0.006906,0.001097,0.000977,0.448927,0.451051,0.500090,0.499910,0.231490,0.204645,0.302936,0.260929,89

In [676]:
target_year_data['REP_VOTES'] = target_year_data['REP_VOTES%'] * target_year_data['COUNTY_TOTALVOTES']
target_year_data['DEM_VOTES'] = target_year_data['DEM_VOTES%'] * target_year_data['COUNTY_TOTALVOTES']

In [677]:
target_year_data['REP_VOTES'] = target_year_data['REP_VOTES'].astype(np.int64)
target_year_data['DEM_VOTES'] = target_year_data['DEM_VOTES'].astype(np.int64)

In [678]:
target_year_data.loc[(data_2020['STATE_FIPS'] == 6) ]

,Unnamed: 0,YEAR_FIPS,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES%,DEM_VOTES%,REP_VOTES,DEM_VOTES
187,506,20165147,5,147,1,1,5.5,33108,7,11,1013,879,50,59,16,11,5,1,3598,3276,6874,2502,2314,1657,821,836,1418,704,714,1819,885,934,1980,866,1114,0.001018,0.001600,0.147367,0.127873,0.007274,0.008583,0.002328,0.001600,0.000727,0.000145,0.363980,0.336631,0.523422,0.476578,0.241053,0.206285,0.264620,0.288042,3260,1,0.634373,0.365627,2068,1191
188,507,20165149,5,149,1,1,4.5,30690,167,149,202,214,2042,2209,101,100,5,9,10888,10886,21774,10281,10258,6010,3106,2904,5065,2619,2446,5803,2891,2912,4896,2270,2626,0.007670,0.006843,0.009277,0.009828,0.093782,0.101451,0.004639,0.004593,0.000230,0.000413,0.472169,0.471112,0.500046,0.499954,0.276017,0.232617,0.266511,0.224855,8423,1,0.634002,0.365998,5340,3082
189,508,20166001,6,1,0,-1,4.3,71347,243563,223223,103211,91757,178801,183447,8672,9191,8112,7584,820526,788148,1608674,414415,415670,384662,195927,188735,486877,242200,244677,448057,220816,227241,289078,129205,159873,0.151406,0.138762,0.064159,0.057039,0.111148,0.114036,0.005391,0.005713,0.005043,0.004714,0.257613,0.258393,0.510064,0.489936,0.239117,0.302657,0.278526,0.179700,607664,0,0.332392,0.667608,201982,405681
190,509,20166003,6,3,0,-1,6.3,43438,11,7,1,1,45,54,137,124,0,0,529,553,1082,364,399,246,120,126,184,99,85,329,173,156,323,161,162,0.010166,0.006470,0.000924,0.000924,0.041590,0.049908,0.126617,0.114603,0.000000,0.000000,0.336414,0.368762,0.488909,0.511091,0.227357,0.170055,0.304067,0.298521,7262,1,0.623586,0.376414,4528,2733
191,510,20166005,6,5,0,-1,5.9,43648,276,246,90,678,1820,2886,336,464,43,44,17226,19512,36738,15876,17492,6280,3208,3072,7241,4264,2977,10581,5916,4665,12636,6124,6512,0.007513,0.006696,0.002450,0.018455,0.049540,0.078556,0.009146,0.012630,0.001170,0.001198,0.432141,0.476128,0.468888,0.531112,0.170940,0.197098,0.288012,0.343949,18969,1,0.622555,0.377445,11809,7159
192,511,20166007,6,7,0,-1,6.6,120924,5057,5066,1646,2170,16876,17503,2839,2739,301,309,112975,110595,223570,98312,95692,53375,27200,26175,65353,33802,31551,52232,25656,26576,52610,23937,28673,0.022619,0.022660,0.007362,0.009706,0.075484,0.078289,0.012698,0.012251,0.001346,0.001382,0.439737,0.428018,0.505323,0.494677,0.238740,0.292316,0.233627,0.235318,89168,0,0.385123,0.614877,34340,54827
193,512,20166009,6,9,0,-1,5.7,39007,360,281,151,261,2405,2602,404,411,47,48,22321,22352,44673,20581,20574,8800,4593,4207,7770,4059,3711,12534,6088,6446,15569,7612,7957,0.008059,0.006290,0.003380,0.005842,0.053836,0.058245,0.009043,0.009200,0.001052,0.001074,0.460703,0.460547,0.499653,0.500347,0.196987,0.173931,0.280572,0.348510,20618,1,0.622687,0.377313,12838,7779
194,513,20166011,6,11,0,-1,15.6,40410,202,175,113,142,5828,6452,269,305,60,60,10338,10860,21198,9463,9941,6551,3437,3114,5477,2848,2629,5176,2661,2515,3994,1914,2080,0.009529,0.008255,0.005331,0.006699,0.274932,0.304368,0.012690,0.014388,0.002830,0.002830,0.446410,0.468959,0.487688,0.512312,0.309039,0.258373,0.244174,0.188414,3198,1,0.624593,0.375407,1997,1200
195,514,20166013,6,13,0,-1,4.5,66525,95645,85042,56428,49586,138386,139323,5337,5622,3274,3056,568023,541335,1109358,378808,370358,287463,146786,140677,279081,138847,140234,319529,155661,163868,223285,100041,123244,0.086217,0.076659,0.050865,0.044698,0.124744,0.125589,0.004811,0.005068,0.002951,0.002755,0.341466,0.3338

In [679]:
#save to csv
target_year_data.to_csv("../../data/F20_phase3_0717_target.csv")